In [ ]:
# import packages
import cozmo
from cozmo.world import EvtNewCameraImage
import numpy
import time
from cozmo.util import ImageBox, Rotation, degrees, speed_mmps, distance_mm

# Cozmo can see or not: camera is on (True) or off (False)
camera = False
# Cozmo is taking a series of pictures which have to be named and saved in a folder
imageNumber = 0
saved_images_path = '.'
imagename = 'Letter_X'

# Reset Cozmo's body to a neutral position
def tuck_your_shirt_cozmo(robot: cozmo.robot.Robot):
    ''' lift must be down. lift can be 32 to 92mm above the ground'''
    robot.set_lift_height(0.0).wait_for_completed()
    ''' head should look straight ahead. Cozmo looks down at -25 and looks up at 44.5'''
    robot.set_head_angle(degrees(13.0)).wait_for_completed()

# Cozmo takes photos
def collect_images(robot: cozmo.robot.Robot):
    ''' Cozmo's backpack lights are blue before taking pictures'''
    robot.set_all_backpack_lights(cozmo.lights.blue_light)
    time.sleep(2)
    ''' Cozmo is placed at 30 cm of the object and takes his first videoshot'''         
    ''' first Turn on Cozmo's camera'''
    global camera
    camera = True
    time.sleep(2)
    ''' second Cozmo moves forward 20cm'''
    robot.drive_straight(distance_mm(200), speed_mmps(150), False,False,0).wait_for_completed()
    ''' third Cozmo moves backward to its initial location'''
    robot.drive_straight(distance_mm(-200), speed_mmps(150), False,False,0).wait_for_completed()
    ''' fourth save and Turn off Cozmo's camera'''
    latest_image = robot.world.latest_image.raw_image
    latest_image.convert('L').save(saved_images_path+'image.jpeg')
    camera = False
    time.sleep(2)
    ''' last Cozmo gets into the next position. Left Turns are positive. Right turns are negative.'''
    robot.turn_in_place(degrees(-90)).wait_for_completed()
    robot.drive_straight(distance_mm(350), speed_mmps(150), False,False,0).wait_for_completed()
    robot.turn_in_place(degrees(90)).wait_for_completed()
    robot.drive_straight(distance_mm(350), speed_mmps(150), False,False,0).wait_for_completed()
    robot.turn_in_place(degrees(90)).wait_for_completed()
    
   
    
def on_new_camera_image(evt, **kwargs):
    global camera
    if camera:
        pilImage = kwargs['image'].raw_image
        global imagename
        pilImage.save(saved_images_path+f"\\{imagename}-{kwargs['image'].image_number}.jpeg", "JPEG")
        
# Cozmo collects images moving forward and back
def cozmo_collects_images(robot: cozmo.robot.Robot):
    ''' Cozmo gets ready'''
    tuck_your_shirt_cozmo(robot)
    ''' This event handler gets Cozmo interested in collecting images'''
    robot.add_event_handler(cozmo.world.EvtNewCameraImage, on_new_camera_image)
    ''' Cozmo takes pictures and go around the object 3 times doing so'''
    for number_of_pictures in range(11):
        collect_images(robot)
    ''' turn off backpack lights when done'''
    robot.set_all_backpack_lights(cozmo.lights.off_light)
    time.sleep(2)

# Run cozmo_collect_images
cozmo.run_program(cozmo_collects_images, use_viewer = True, force_viewer_on_top = True)